In [9]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.python.framework import graph_util

def _parse_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    features = tf.parse_single_example(example_proto, feature_description)
    data = {}
    image = tf.cast(tf.reshape(features['image'], [28, 28, 1]), dtype=tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [-1,784])
    label = tf.one_hot(features['label'], depth=11, on_value=1)
    label = tf.cast(label, tf.float32)
    label = tf.reshape(label, [-1,11])
    return image, label

feature_description = {
    'image': tf.FixedLenFeature([28 * 28], tf.int64),
    'label': tf.FixedLenFeature([], tf.int64),
    'shape': tf.FixedLenFeature([3], tf.int64, default_value=[28, 28, 1])
}

nist_train = tf.data.TFRecordDataset('./nist_train.tfrecord').map(_parse_function).shuffle(120000).batch(32).repeat()
nist_test = tf.data.TFRecordDataset('./nist_test.tfrecord').map(_parse_function).shuffle(20000).batch(10000)

iterator = nist_train.make_one_shot_iterator()
test_iterator = nist_test.make_one_shot_iterator()


data_element = iterator.get_next()
test_element = test_iterator.get_next()


sess = tf.InteractiveSession()

D:\anaconda\envs\tfenv\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 11])
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [5]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Instructions for updating:
Colocations handled automatically by placer.


W0509 21:53:46.901336  8084 deprecation.py:323] From D:\anaconda\envs\tfenv\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 11])
b_fc2 = bias_variable([11])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0509 21:53:46.944130  8084 deprecation.py:506] From <ipython-input-6-ec4765b33565>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.global_variables_initializer().run()
for i in range(20000):
    image, label = sess.run(data_element)
    image = np.reshape(image, [-1,784])
    label = np.reshape(label, [-1,11])
    
    tx, ty = sess.run(test_element)
    tx = np.reshape(tx, [-1,784])
    ty = np.reshape(ty, [-1,11])
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:tx, y_:ty , keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x: image, y_: label, keep_prob: 0.5})



ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Conv2D (defined at <ipython-input-3-4508c1a3afaf>:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'Conv2D', defined at:
  File "D:\anaconda\envs\tfenv\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anaconda\envs\tfenv\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\anaconda\envs\tfenv\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\anaconda\envs\tfenv\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "D:

In [2]:
batch = mnist.test.next_batch(2000)

NameError: name 'nist_train' is not defined